## Семинар 12

# Тема: Кросс-валидация

**Разбиение выборки** - это разделение имеющихся данных на несколько частей для проведения процессов обучения и оценки качества алгоритма машинного обучения на полностью независимых наборах данных.

### Способы разбиения выборки
1. **hold-out** разбиение (отложенная выборка, т.е. на две части - обучающую и тестовую)
2. **cross-validation** разбиение (на k наборов, имеющих обучающую и тестовую части)

    #### Виды разбиений для перекрёстной прорверки:
    1. **k-fold** (разбиение на k частей с примерно одинаковым количеством объектов в каждом, из которых получают k наборов обучающих и
    тестовых выборок, в которых одна часть выступает в роли тестовой выборки, а объединение остальных — в роли обучающей)
    2. **Stratified k-fold**(k-fold разбиение со стратификацией, т.е. все наборы содержат примерно такой же процент объектов каждого
    класса, что и исходные данные)
    2. **leave-one-out** (отложенный пример)
    и др.

**Кросс-валидация** (перекрёстная проверка) — это процедура для оценки обобщающей способности алгоритмов, состоящая из следующих этапов:

1. Фиксируется число блоков k. 

2. Выборка разбивается на k блоков определённым образом и получается k наборов данных, в которых один блок выступает в роли тестовой выборки, а объединение остальных блоков — в роли тренировочной. 

3. Осуществляется k итераций обучения модели на тренировочной выборке и подсчёт метрик на тестовой.

4. Итоговые метрики получаются усреднением k получившихся результатов.

Импортируем необходимые библиотеки:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, r2_score, root_mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.datasets import load_iris, load_wine, load_diabetes

from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut, cross_val_score, cross_validate

Загрузим встроенные данные об ирисах и выведем их описание:

In [2]:
iris = load_iris()
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

Выделим признаки и целевую переменную:

In [3]:
X = iris.data
y = iris.target

Сделаем hold-out разбиение данных на обучающую и тестовую части:

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Обучим модель логистической регрессии на обучающих данных и найдём значение метрики accuracy на тестовых.

In [5]:
model = LogisticRegression()
model.fit(X_train, y_train)
print('accuracy =', model.score(X_test, y_test))

accuracy = 1.0


Поскольку accuracy = 1, предполагаем, что модель всегда верно классифицирует данные. Проверим так ли это, проведя перекрёстную проверку.

Сделаем разбиение исходных данных на k-блоков с перемешиванием, где k = 3. Выведем индексы обучающих и тестовых частей каждого блока, а также их длины.

In [6]:
kf = KFold(n_splits = 3,shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Fold: {}, Длинна train: {}, Длинна test: {}".format(i, len(train_index), len(test_index)))
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold: 0, Длинна train: 100, Длинна test: 50
  Train: index=[  0   1   2   3   5   6   7   8  13  14  17  20  21  23  24  25  28  33
  34  35  37  38  39  40  41  43  44  46  47  48  49  50  52  53  54  57
  58  59  60  61  62  63  66  67  70  71  72  74  77  79  80  83  84  87
  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 105 106
 107 111 112 113 114 115 116 117 119 120 121 122 123 124 125 126 129 130
 134 135 136 138 139 140 144 147 148 149]
  Test:  index=[  4   9  10  11  12  15  16  18  19  22  26  27  29  30  31  32  36  42
  45  51  55  56  64  65  68  69  73  75  76  78  81  82  85  86 104 108
 109 110 118 127 128 131 132 133 137 141 142 143 145 146]
Fold: 1, Длинна train: 100, Длинна test: 50
  Train: index=[  1   2   3   4   6   9  10  11  12  14  15  16  17  18  19  20  21  22
  26  27  29  30  31  32  36  37  38  41  42  45  46  48  50  51  52  54
  55  56  57  58  59  61  63  64  65  68  69  71  72  73  74  75  76  78
  79  81  82  85  86  87  88  90  91 

Сделаем k-блочную перекрёстную проверку (k = 3) модели логистической регрессии для метрики accuracy, используя библиотечную функцию cross_val_score.

In [7]:
cv_results = cross_val_score(model,                  # модель
                             X,                      # матрица признаков
                             y,                      # вектор цели
                             cv = kf,                # тип разбиения (можно указать просто число фолдов cv = 3)
                             scoring = 'accuracy',   # метрика 'f1_macro','precision_macro', 'recall_macro'
                                                             # 'f1_micro','precision_micro', 'recall_micro'
                                                             # 'f1_weighted','precision_weighted', 'recall_weighted'
                             n_jobs=-1)              # используются все ядра CPU
cv_results

array([1.  , 0.92, 0.98])

Найдём среднее по кросс-валидации:

In [8]:
cv_results.mean()

0.9666666666666667

Делаем вывод, что модель классифицирует верно только 97% данных.

# Кросс-валидация в задаче классификации

### 1. Загрузите встроенный датасет o классификации вина из библиотеки sklearn.datasets. Выведите его описание. Обозначьте за _X_ двумерный numpy-массив признаков, а за _X_frame_ датафрейм признаков с соответствующими названиями колонок. Выведите размеры и первые пять строк, полученного датафрейма. Обозначьте за _y_ одномерный numpy-массив, а за _y_series_  - серию со значениями целевой переменной. Определите количество классов и количество объектов в каждом классе для целевой переменной. Убедитесь в том, что в данных нет пропущенных значений и они все числовые.
Указание: Для загрузки датасета [wine](https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.load_wine.html) напишите wine = load_wine(). Для определения количества объектов в каждом классе используйте метод _.value_counts()_ библиотеки _pandas_.

### 2. Сделайте hold-out (на две части) разбиение данных на обучающую и тестовую части в соотношении 4:1 тремя способами: без перемешивания и с перемешиванием двумя разными способами. Выведите количество объектов, попавшых в обучающую и в тестовую части. Выведите на экран индексы объектов обучающей и тестовой частей для каждого разбиения.
Указание: Используйте функцию train_test_split. Для разбиение без перемешивания, укажите параметр shuffle=False. Для разбиений с перемешиванием, зафиксируйте разные способы перемешивания, указав random_state=0 и  random_state=1. Для выведения индексов объектов, попавших в обучающую и тестовую выборки, передавайте в функцию датафрейм и серию (не numpy-массивы), а индексы выводите при помощи .index библиотеки pandas.

### 3.Обучите модель логистической регрессии LogisticRegression(max_iter=5000) на трёх наборах обучающих данных, полученных в предыдущем задании. Сделайте предсказания на соответствующих трёх наборах тестовых данных и вычислите значения метрики accuracy для каждого из наборов. Сделайте вывод о том, влияет ли способ разбиения на значение метрики.

### 4. Сделайте разбиение исходных данных на k-блоков с перемешиванием, где k = 5. Выведите индексы обучающих и тестовых частей каждого блока.

Указание: Используйте класс [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) с параметрами n_splits = 5, shuffle=True, random_state=42.

### 5. Напишите свой код, осуществляющий k-блочную перекрёстную проверку (кросс-валидацию) с количеством блоков k = 5 модели логистической регрессии для метрики accuracy. Усредните полученный результат.

Указание: Используйте индексы блоков, полученных в предыдущем задании.

### 6. Сделайте k-блочную перекрёстную проверку с k = 5 модели логистической регрессии для метрики accuracy, используя библиотечную функцию cross_val_score. Убедитесь, что получится тот же результат, что и в предыдущем задании. Также выведите дисперсию по кросс-валидации.
Указание: Функция [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) позволяет осуществлять перекрёстную проверку для вычисления только одной метрики, которую нужно указать в параметре scoring.

### 7. Сделайте k-блочную перекрёстную проверку с k = 5 модели логистической регрессии для двух метрик accuracy и F1-мера, используя библиотечную функцию cross_validate. Выведите полученные массивы со значениями метрик на обучающих и тестовых данных. Усредните их значения.
Указание: Функция [cross_validate](https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.cross_validate.html) позволяет осуществлять перекрёстную проверку нескольких метрик, которые нужно указать списком в параметре scoring.

### 8. Сделайте разбиение исходных данных на k-блоков со стратификацией, число блоков возьмите равным 5. Выведите индексы обучающих и тестовых частей каждого блока. Выведите для полученных блоков обучающих данных количество объектов каждого класса, сравните результаты с количеством объектов каждого класса для целевой переменной в целом.

Указание: Используйте класс [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) с параметрами n_splits = 5, shuffle=True, random_state=42. StratifiedKFold возвращает стратифицированные фолды, т.е. каждый блок содержит примерно такой же процент объектов каждого класса, что и полный набор.

### 9. Сделайте k-блочную перекрёстную проверку (k = 5) со стратификацией модели логистической регрессии для двух метрик accuracy и F1-мера, используя библиотечную функцию cross_validate. Выведите полученные массивы со значениями метрик на обучающих и тестовых данных. Усредните их значения.

Указание: Для того, чтобы выводились массивы со значениями метрик на обучающих данных необходимо указать значения параметра return_train_score=True.

### 10. Сделайте leave-one-out разбиение данных. Выведите индексы обучающих и тестовых частей.

Указание: Используйте  класс [LeaveOneOut](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html).

### 11. Сделайте перекрёстную проверку модели логистической регрессии для метрик accuracy и F1-мера, изпользуя leave-one-out разбиение. Выведите полученные массивы со значениями метрик на тестовых данных. Усредните их значения.

### 12. Напишите свой код, осуществляющий k-блочную перекрёстную проверку (k = 5) модели логистической регрессии для метрик accuracy и F1_мера с предварительным масштабированием данных методом стандартной нормализации. Усредните полученные результаты. 

### 13. Сделайте k-блочную перекрёстную проверку (k = 5) модели логистической регрессии для двух метрик accuracy и F1-мера с предварительным масштабированием данных методом стандартной нормализации, используя библиотечную функцию cross_validate. Усредните значения метрик.

Указание: Используйте конвейер, передайте его в функцию cross_validate в качестве параметра model.

# Кросс-валидация в задаче регрессии

### 1. Загрузите встроенный датасет o прогрессировании диабета из библиотеки sklearn.datasets. Выведите его описание. Обозначьте за _X_ двумерный numpy-массив признаков, а за _X_frame_ датафрейм признаков с соответствующими названиями колонок. Выведите размеры и первые пять строк, полученного датафрейма. Обозначьте за _y_ одномерный numpy-массив, а за _y_series_  - серию со значениями целевой переменной. Выведите количество уникальных значений целевой переменной. Убедитесь в том, что в данных нет пропущенных значений и они все числовые.
Указание: Для загрузки датасета [diabetes](https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.load_wine.html) напишите diabetes = load_diabetes(). Для определения количества объектов в каждом классе используйте метод _.value_counts()_ библиотеки _pandas_.

### 2. Сделайте hold-out (на две части) разбиение данных на обучающую и тестовую части в соотношении 4:1, зафиксировав способ перемешивания данных, указав значение параметра random_state=5. Выведите количество объектов, попавших в обучающую и тестовую части. Обучите модель линейной регрессии на обучающих данных и сделайте предсказание на тестовых данных. Вычислите значения метрик коэффициент детерминации и RMSE. На основе полученных значений сделайте предположение о качестве модели.

### 3. Напишите свой код, осуществляющий k-блочную перекрёстную проверку с количеством блоков k = 10 модели линейной регрессии для метрик коэффициент детерминации и RMSE. Усредните полученные результаты. Сделайте окончательный вывод о возможности пименения модели линейной регрессии к этим данным.

### 4. Сделайте k-блочную перекрёстную проверку (k = 10) модели линейной регрессии для двух метрик коэффициент детерминации и ошибка RMSE, используя библиотечную функцию cross_validate. Выведите полученные массивы со значениями метрик и усредните их. Сравните полученные результаты с предыдущим пунктом.

Указание: Укажите параметр scoring = ['r2', 'neg_root_mean_squared_error']

### 5. Вместо модели линейной регрессии, примените полиномиальную модель второй степени. Сделайте k-блочную перекрёстную проверку (k = 10) этой модели для метрик коэффициент детерминации и ошибки RMSE с предварительным масштабированием данных min-max методом, используя библиотечную функцию cross_validate. Усредните полученное значения метрик. Используйте конвейер make_pipeline. Построенная полиномиальная модель описывает данные лучше линейной модели или нет?